## Notebook to compare Monte Carlo Search with Monte Carlo Tree Search

In [ ]:
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import train_test_split

n_samples = 10000
n_features = 16
n_classes = 8
n_labels = 2
random_state = 0

X, Y = make_multilabel_classification(
    n_samples = n_samples,
    n_features = n_features,
    n_classes = n_classes,
    n_labels = n_labels,
    random_state = random_state)

test_size = 0.2
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size, random_state=random_state)

from sklearn.multioutput import ClassifierChain
from sklearn.linear_model import LogisticRegression

solver = "liblinear"
base = LogisticRegression(solver=solver)
chain = ClassifierChain(base)

chain = chain.fit(X_train, Y_train)


## Fixed range for N

In [ ]:
from sklearn.metrics import hamming_loss
from mcts_inference.mcts import MCTS
from mcts_inference.mcts_config import MCTSConfig
from mcts_inference.policy import UCB
from mcts_inference.constraints import Constraint


Ns: list[int] = [100, 200, 400, 800, 1600, 3200, 6400, 12800, 25600]
M: int = 100

losses1 = []
chain_losses = hamming_loss(chain.predict(X_test[:M]), Y_test[:M])

print("> n: ",end="")
for n in Ns:
    print(n, end=", ")
    constraint = Constraint(max_iter=True, n_iter=n)

    config1 = MCTSConfig(n_classes, selection_policy=UCB(), constraint=constraint, step_once=True)

    out1 = MCTS(X_test[:M], chain, config1)

    losses1.append(hamming_loss(out1, Y_test[:M]))

import matplotlib.pyplot as plt

fig = plt.figure(figsize=(10, 5))

plt.plot(Ns, losses1, label="MCTS UCB", marker="o", markersize=3)
plt.plot(Ns, [chain_losses]*len(Ns), label="Chain", marker="o", markersize=1)

plt.xscale("log")
plt.xlabel("Number of Log Iterations")
plt.ylabel("Hamming Loss")

plt.title(f"Hamming Loss vs Number of log Iterations, L={n_classes}")
plt.legend()
plt.show();

## Wider range for N

In [ ]:
from sklearn.metrics import hamming_loss
from mcts_inference.mcts import MCTS
from mcts_inference.mcts_config import MCTSConfig
from mcts_inference.policy import UCB
from mcts_inference.constraints import Constraint

Ns = [x for x in range(100, 16000, 800)]
losses1 = []
chain_losses = hamming_loss(chain.predict(X_test[:M]), Y_test[:M])

print("> n: ",end="")
for n in Ns:
    print(n, end=", ")
    constraint = Constraint(max_iter=True, n_iter=n)

    config1 = MCTSConfig(n_classes, selection_policy=UCB(), constraint=constraint, step_once=True)

    out1 = MCTS(X_test[:M], chain, config1)

    losses1.append(hamming_loss(out1, Y_test[:M]))

import matplotlib.pyplot as plt

fig = plt.figure(figsize=(10, 5))

plt.plot(Ns, losses1, label="MCTS UCB", marker="o", markersize=3)
plt.plot(Ns, [chain_losses]*len(Ns), label="Chain", marker="o", markersize=1)

plt.xlabel("Number of Iterations")
plt.ylabel("Hamming Loss")

plt.title(f"Hamming Loss vs Number of Iterations, L={n_classes}")
plt.legend()
plt.show();

## Just MCTS with few Iterations

In [ ]:
from sklearn.metrics import hamming_loss
from mcts_inference.mcts import MCTS
from mcts_inference.mcts_config import MCTSConfig
from mcts_inference.policy import UCB
from mcts_inference.constraints import Constraint

Ns = [x for x in range(100, 2000, 50)]
M = 100

losses4 = []
chain_losses: float = hamming_loss(chain.predict(X_test[:M]), Y_test[:M])

print("> n: ",end="")
for n in Ns:
    print(n, end=", ")
    constraint = Constraint(max_iter=True, n_iter=n)

    config4 = MCTSConfig(n_classes, exploration_policy=UCB(), constraint=constraint, step_once=True)

    avg = 10
    loss = 0
    for i in range(avg):
        out4 = MCTS(X_test[:M], chain, config4)
        loss += hamming_loss(out4, Y_test[:M])

    losses4.append(loss/avg)

import matplotlib.pyplot as plt

fig = plt.figure(figsize=(10, 5))

plt.plot(Ns, losses4, label="MCTS", marker="o", markersize=3)
plt.plot(Ns, [chain_losses]*len(Ns), label="Chain", marker="o", markersize=1)

plt.xlabel("Number of Iterations")
plt.ylabel("Hamming Loss")

plt.title(f"Hamming Loss vs Number of Iterations, L={n_classes}, Averaged over {avg} runs")
plt.legend()
plt.show();

In [ ]:
from sklearn.metrics import hamming_loss
from mcts_inference.mcts import MCTS
from mcts_inference.mcts_config import MCTSConfig
from mcts_inference.policy import UCB
from mcts_inference.constraints import Constraint


NNs = [x for x in range(2000, 2500, 50)]
M = 100

losses5 = []
chain_losses = hamming_loss(chain.predict(X_test[:M]), Y_test[:M])

print("> n: ",end="")
for n in Ns:
    print(n, end=", ")
    constraint = Constraint(max_iter=True, n_iter=n)

    config5 = MCTSConfig(n_classes, exploration_policy=UCB(), constraint=constraint, step_once=True)

    avg = 10
    loss = 0
    for i in range(avg):
        out5 = MCTS(X_test[:M], chain, config5)
        loss += hamming_loss(out5, Y_test[:M])

    losses5.append(loss/avg)

import matplotlib.pyplot as plt

fig = plt.figure(figsize=(10, 5))

plt.plot(NNs, losses5, label="MCTS", marker="o", markersize=3)
plt.plot(NNs, [chain_losses]*len(NNs), label="Chain", marker="o", markersize=1)

plt.xlabel("Number of Iterations")
plt.ylabel("Hamming Loss")

plt.title(f"Hamming Loss vs Number of Iterations, L={n_classes}, Averaged over {avg} runs")
plt.legend()
plt.show();

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(10, 5))

plt.plot(Ns+NNs, losses4+losses5, label="MCTS", marker="o", markersize=3)
plt.plot(Ns+NNs, [chain_losses]*len(Ns+NNs), label="Chain", marker="o", markersize=1)

plt.xlabel("Number of Iterations")
plt.ylabel("Hamming Loss")

plt.title(f"Hamming Loss vs Number of Iterations, L={n_classes}, Averaged over {avg} runs")
plt.legend()
plt.show();